In [114]:
import requests
import pandas as pd
import plotly.express as px
import calendar

In [115]:
def collect_data(link):
        return (pd.DataFrame((requests.get(link).json())))
def plotly_chart(df, yvalue, xvalue, text, title, y_title, x_title, color=None, discrete_sequence=None, map=None):
        fig = px.bar(df, y=yvalue,
                     x=xvalue, text=text, color=color, color_discrete_sequence=discrete_sequence, color_discrete_map=map)
        fig.update_layout(
            title=title,
            yaxis_title=y_title,
            xaxis_title=x_title,
        )
        fig.update_traces(textposition='outside')
        return fig

In [116]:


users = collect_data('https://vietop.tech/api/get_data/users')
order_details = collect_data('https://vietop.tech/api/get_data/order_details')


In [123]:
order_details['created_at'] = order_details['created_at'].astype('datetime64[ns]')
order_details['created_at_month'] = order_details['created_at'].dt.month

order_details['created_at_year'] = order_details['created_at'].dt.year
df = order_details[['branch_id', 'hv_id', 'ec_id', 'detail_price', 'created_at', 'created_at_month', 'created_at_year']].query("created_at > '2023-01-01'")
df_ec = df.groupby(['ec_id', 'created_at_month'],as_index = False).agg({'hv_id':'count','detail_price': 'mean'})

def process_before_plot(df):
    df['detail_price'] = round(df['detail_price']/ 1000, 0)
    df.rename(columns={'hv_id':'count_hv', 'detail_price':'average_price'},inplace=True)
    df = df.merge(users[['fullname', 'id']], left_on = 'ec_id', right_on = 'id', how = 'left', validate='many_to_one').sort_values("created_at_month")
    # Filter EC
    df = df.query('id == 250')[['created_at_month', 'count_hv','average_price']]
    df['text'] = df['average_price'].apply(lambda x: f'{x:,.0f}').astype(str) + "k (" + df['count_hv'].astype(str) + " students) "
    # Convert numeric months to text representation
    df['created_at_month'] = [calendar.month_name[month] for month in df['created_at_month']]
    return df
df_ec = process_before_plot(df_ec)

In [125]:
# Assuming your DataFrame is named df
fig = plotly_chart(df_ec, xvalue='created_at_month', yvalue='average_price',
             title='Average Sales by Month',
             x_title='Month',
             y_title='Average Sales',
             text='text')
fig.update_traces(
    hovertemplate="Trung bình doanh thu: <extra></extra>%{text}")


fig.show()